Cell 1 — Setup + Connessione Qdrant

In [ ]:
import os
import random
import numpy as np
import pandas as pd

from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter

# --- Config (adatta se serve) ---
QDRANT_HOST = os.getenv("QDRANT_HOST", "127.0.0.1")
QDRANT_PORT = int(os.getenv("QDRANT_PORT", "6333"))
QDRANT_COLLECTION = os.getenv("QDRANT_COLLECTION", "financial_docs")

client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

# sanity check
info = client.get_collection(QDRANT_COLLECTION)
info


Cell 2 — Utilities: campionamento embedding da Qdrant

Questa cell preleva un campione casuale di N vettori dalla collection (via scroll), con possibilità di filtro.

In [ ]:
def sample_vectors_from_qdrant(
    client: QdrantClient,
    collection: str,
    n: int = 30,
    qdrant_filter: Filter | None = None,
    seed: int = 42,
    scroll_page_size: int = 256,
    max_scroll_pages: int = 200,
    with_payload: bool = True,
):
    """
    Campiona n punti dalla collection Qdrant usando scroll.
    Nota: scroll non è 'random' nativo; qui raccogliamo un pool e poi campioniamo.
    """
    rng = random.Random(seed)

    points_pool = []
    offset = None
    pages = 0

    while pages < max_scroll_pages and len(points_pool) < max(n * 10, 300):
        page, offset = client.scroll(
            collection_name=collection,
            scroll_filter=qdrant_filter,
            limit=scroll_page_size,
            with_vectors=True,
            with_payload=with_payload,
            offset=offset,
        )
        pages += 1
        if not page:
            break
        points_pool.extend(page)

        if offset is None:
            break

    if len(points_pool) < n:
        raise ValueError(f"Pool troppo piccolo: trovati {len(points_pool)} punti, richiesti {n}.")

    sampled = rng.sample(points_pool, n)

    # vettori in matrice NxD
    X = np.array([p.vector for p in sampled], dtype=np.float32)

    # payload opzionale (utile per debug)
    payloads = [p.payload if with_payload else None for p in sampled]
    ids = [p.id for p in sampled]

    return X, ids, payloads


# --- ESEMPIO: campiona 30 vettori senza filtri ---
X, ids, payloads = sample_vectors_from_qdrant(client, QDRANT_COLLECTION, n=30, seed=7)
X.shape, ids[:3]


Cell 2-bis — Normalizzazione L2 (NUOVA)

In [ ]:
def l2_normalize(X, eps=1e-12):
    X = np.asarray(X, dtype=np.float64)
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    return X / (norms + eps)

# embedding originali (RAW)
X_raw = X.copy()

# embedding normalizzati (L2)
X_l2 = l2_normalize(X)


Cell 3 — Distanze (coseno, euclidea, chebyshev, minkowski, manhattan)

Implementazione solo NumPy (niente SciPy richiesto).

In [ ]:
def pairwise_distance_matrix(X: np.ndarray, metric: str, p: float = 3.0, eps: float = 1e-12) -> np.ndarray:
    """
    Ritorna matrice NxN delle distanze per la metrica richiesta.
    metric: 'cosine' | 'euclidean' | 'chebyshev' | 'minkowski' | 'manhattan'
    p: usato solo per minkowski
    """
    X = np.asarray(X, dtype=np.float64)
    n = X.shape[0]

    if metric == "cosine":
        # cosine distance = 1 - cosine similarity
        norms = np.linalg.norm(X, axis=1, keepdims=True) + eps
        Xn = X / norms
        sim = Xn @ Xn.T
        D = 1.0 - sim
        np.fill_diagonal(D, 0.0)
        return D

    # per Lp: usa broadcasting Nx1xD - 1xNxD
    diff = X[:, None, :] - X[None, :, :]

    if metric == "euclidean":
        D = np.sqrt(np.sum(diff * diff, axis=2))
    elif metric == "manhattan":
        D = np.sum(np.abs(diff), axis=2)
    elif metric == "chebyshev":
        D = np.max(np.abs(diff), axis=2)
    elif metric == "minkowski":
        D = np.sum(np.abs(diff) ** p, axis=2) ** (1.0 / p)
    else:
        raise ValueError(f"Metrica non supportata: {metric}")

    np.fill_diagonal(D, 0.0)
    return D


Cell 4 — Indici di omogeneità (compattezza) del campione

Qui calcoliamo più indicatori per ogni metrica:

A) Pairwise dispersion (solo distanze tra punti)

mean_pairwise, median_pairwise

std_pairwise, cv_pairwise (std/mean)

iqr_pairwise (robustezza)

p95_pairwise, max_pairwise

%_under_median+MAD (proxy di “densità” robusta)

B) Centroid dispersion (distanza di ogni punto dal centroide)

mean_to_centroid, std_to_centroid, p95_to_centroid, max_to_centroid

radius_ratio = p95/max (più vicino a 1 ⇒ coda meno estrema; più basso ⇒ outlier)

def robust_stats_1d(v: np.ndarray) -> dict:
    v = np.asarray(v, dtype=np.float64)
    v = v[np.isfinite(v)]
    if v.size == 0:
        return {}

    q25, q50, q75 = np.percentile(v, [25, 50, 75])
    iqr = q75 - q25
    mad = np.median(np.abs(v - q50))  # Median Absolute Deviation

    return {
        "count": int(v.size),
        "mean": float(np.mean(v)),
        "median": float(q50),
        "std": float(np.std(v, ddof=1)) if v.size > 1 else 0.0,
        "min": float(np.min(v)),
        "max": float(np.max(v)),
        "p95": float(np.percentile(v, 95)),
        "iqr": float(iqr),
        "mad": float(mad),
    }


def homogeneity_indices(X: np.ndarray, metric: str, p: float = 3.0) -> dict:
    """
    Calcola indici di omogeneità/compattezza per un campione di embedding.
    """
    D = pairwise_distance_matrix(X, metric=metric, p=p)
    n = D.shape[0]

    # estrai solo triangolo superiore (distanze uniche)
    tri = D[np.triu_indices(n, k=1)]
    pw = robust_stats_1d(tri)

    # centroid dispersion
    C = np.mean(X, axis=0, keepdims=True)
    Dc = pairwise_distance_matrix(np.vstack([X, C]), metric=metric, p=p)  # (n+1)x(n+1)
    dist_to_centroid = Dc[:-1, -1]
    cd = robust_stats_1d(dist_to_centroid)

    # indicatori sintetici
    mean_pw = pw["mean"]
    std_pw = pw["std"]
    cv_pw = (std_pw / mean_pw) if mean_pw > 0 else np.nan

    # proxy densità robusta: quota sotto (median + MAD)
    thr = pw["median"] + pw["mad"]
    dense_ratio = float(np.mean(tri <= thr))

    radius_ratio = (cd["p95"] / cd["max"]) if cd["max"] > 0 else np.nan

    return {
        "metric": metric,
        "minkowski_p": p if metric == "minkowski" else None,

        # Pairwise
        "mean_pairwise": mean_pw,
        "median_pairwise": pw["median"],
        "std_pairwise": std_pw,
        "cv_pairwise": cv_pw,
        "iqr_pairwise": pw["iqr"],
        "p95_pairwise": pw["p95"],
        "max_pairwise": pw["max"],
        "dense_ratio_(<=median+MAD)": dense_ratio,

        # To centroid
        "mean_to_centroid": cd["mean"],
        "std_to_centroid": cd["std"],
        "p95_to_centroid": cd["p95"],
        "max_to_centroid": cd["max"],
        "radius_ratio_(p95/max)": radius_ratio,
    }


Cell 5 — Esecuzione confronto metriche + tabella finale

rows = []

# 🔹 TEST SU EMBEDDING RAW
for m, p in metrics:
    if m == "minkowski":
        r = homogeneity_indices(X_raw, metric=m, p=p)
    else:
        r = homogeneity_indices(X_raw, metric=m)
    r["vector_space"] = "raw"
    rows.append(r)

# 🔹 TEST SU EMBEDDING L2-NORMALIZZATI
for m, p in metrics:
    if m == "minkowski":
        r = homogeneity_indices(X_l2, metric=m, p=p)
    else:
        r = homogeneity_indices(X_l2, metric=m)
    r["vector_space"] = "l2"
    rows.append(r)

df = pd.DataFrame(rows)


# ranking: più basso = più omogeneo (a parità di interpretazione)
rank_cols = ["mean_pairwise", "median_pairwise", "p95_pairwise", "mean_to_centroid", "p95_to_centroid"]
for c in rank_cols:
    df[f"rank_{c}"] = df[c].rank(method="min", ascending=True)

df.sort_values("rank_mean_pairwise")


In [ ]:
Cell 6 — Indici comparativi tra metriche (accordo tra matrici di distanza)

Per capire quale distanza è “più stabile/affidabile” anche in senso comparativo, è utile misurare quanto cambia la struttura relativa del campione tra metriche:

Correlazione Spearman tra le distanze pairwise (ranking delle coppie)

Stress (MDS-like): quanto le distanze “deviamo” dalla metrica di riferimento dopo una semplice normalizzazione (z-score)

KNN-overlap: quanto sono consistenti i vicini più prossimi al variare della metrica (molto utile per retrieval)

In [ ]:
def spearman_corr(a: np.ndarray, b: np.ndarray) -> float:
    # Spearman = Pearson sui ranghi
    a = np.asarray(a)
    b = np.asarray(b)
    ra = pd.Series(a).rank().to_numpy()
    rb = pd.Series(b).rank().to_numpy()
    ra = ra - ra.mean()
    rb = rb - rb.mean()
    denom = (np.sqrt(np.sum(ra**2)) * np.sqrt(np.sum(rb**2)))
    return float(np.sum(ra * rb) / denom) if denom > 0 else np.nan


def zscore(v: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    v = np.asarray(v, dtype=np.float64)
    mu = np.mean(v)
    sd = np.std(v) + eps
    return (v - mu) / sd


def knn_sets(D: np.ndarray, k: int = 5):
    # ritorna lista di set dei k vicini per ogni punto
    n = D.shape[0]
    out = []
    for i in range(n):
        idx = np.argsort(D[i])
        idx = idx[idx != i][:k]
        out.append(set(idx.tolist()))
    return out


def knn_overlap_score(D1: np.ndarray, D2: np.ndarray, k: int = 5) -> float:
    A = knn_sets(D1, k=k)
    B = knn_sets(D2, k=k)
    overlaps = []
    for s1, s2 in zip(A, B):
        overlaps.append(len(s1.intersection(s2)) / k)
    return float(np.mean(overlaps))


# --- costruiamo tutte le matrici di distanza ---
X_for_comparison = X_l2
dist_mats = {}
for m, p in metrics:
    key = f"{m}(p={p})" if m == "minkowski" else m
    dist_mats[key] = pairwise_distance_matrix(X_for_comparison, metric=m, p=(p or 3.0))


# --- scegli una "reference" (spesso cosine, se la retrieval in Qdrant usa cosine) ---
ref_key = "cosine"
Dref = dist_mats[ref_key]
n = Dref.shape[0]
ref_tri = Dref[np.triu_indices(n, 1)]

comp_rows = []
for key, D in dist_mats.items():
    tri = D[np.triu_indices(n, 1)]

    # Spearman vs ref (confronto del ranking delle coppie)
    sp = spearman_corr(ref_tri, tri)

    # Stress: distanza tra versioni z-scored (non perfetto, ma indicatore pratico)
    stress = float(np.sqrt(np.mean((zscore(ref_tri) - zscore(tri))**2)))

    # KNN overlap
    knn5 = knn_overlap_score(Dref, D, k=5)
    knn10 = knn_overlap_score(Dref, D, k=10)

    comp_rows.append({
        "metric": key,
        "spearman_vs_ref": sp,
        "stress_vs_ref_(zscore_rmse)": stress,
        "knn_overlap@5_vs_ref": knn5,
        "knn_overlap@10_vs_ref": knn10,
    })

df_comp = pd.DataFrame(comp_rows).sort_values(["spearman_vs_ref", "knn_overlap@10_vs_ref"], ascending=False)
df_comp


Cell 7 — Plot (facoltativo) per visualizzare dispersione per metrica

In [ ]:
import matplotlib.pyplot as plt

def plot_pairwise_distributions(dist_mats: dict):
    n = next(iter(dist_mats.values())).shape[0]
    plt.figure(figsize=(10, 6))
    for name, D in dist_mats.items():
        tri = D[np.triu_indices(n, 1)]
        tri = tri[np.isfinite(tri)]
        plt.hist(tri, bins=25, alpha=0.4, label=name)
    plt.title("Distribuzione distanze pairwise (triangolo superiore)")
    plt.xlabel("distanza")
    plt.ylabel("frequenza")
    plt.legend()
    plt.show()

plot_pairwise_distributions(dist_mats)


Quali “indici di bontà” ti consiglio per scegliere la metrica?

Quando dici “migliore per calcolare l’omogeneità” di 30 embedding, di solito vuoi una metrica che:

separa bene un campione compatto da uno meno compatto
→ usa: mean/median/p95 pairwise, mean/p95 to centroid

non venga dominata da outlier / code estreme
→ usa: IQR, MAD, radius_ratio (p95/max), e confronta p95 vs max

sia stabile rispetto al concetto operativo (retrieval / ranking)
→ usa: KNN overlap e Spearman (se Qdrant lavora in cosine, confronta vs cosine come reference)

Se vuoi un criterio “one-liner” pratico, puoi creare uno score composito (basso=meglio), ad esempio:

score = 0.5 * z(mean_pairwise) + 0.3 * z(p95_pairwise) + 0.2 * z(cv_pairwise)

e poi, separatamente, guardare knn_overlap@10_vs_ref per capire quanto “cambia” la neighborhood.